<a href="https://colab.research.google.com/github/akaanurag281/TheHinduFeedParser/blob/main/PythonTheHinduFeedParse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!pip install feedparser --quiet
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install openai --quiet
print("Completed!")

Completed!


In [31]:
#OPEN AI API and Cosine Similarity Functions + My SQL Server + Embedding by BERT
from openai import OpenAI
import os
import pandas as pd
import numpy as np
os.environ['OPENAI_API_KEY'] = open('/content/drive/MyDrive/openaikey.txt').read().strip()
client = OpenAI()
def openai_generate_summary( #Very Costly
    prompt
):
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        #model="gpt-4",
        max_tokens=800,
        temperature=0.5,
        top_p=0.8,
        frequency_penalty=0.6,
        messages=
       [
         {
          "role": "system",
          "content": "You are a helpful assistant for text summarization.",
         },
         {
          "role": "user",
          "content": f"Detailed summarize this with facts, act, policy,etc {prompt}",
         },
        ],
    )
    return res.choices[0].message.content
def get_embedding(text, model ):#="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   response = client.embeddings.create(input=[text], model=model)
   return response.data[0].embedding
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [32]:
#Import News Content from RSS feed
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
def get_news_content(url):
  response = requests.get(url)

  if response.status_code == 200:
    news_content = response.text
    soup = BeautifulSoup(news_content, 'html.parser')
    content_element = soup.find('div', {'class': 'articlebodycontent', 'itemprop': 'articleBody'})
    if content_element:
      news_content = content_element.get_text()
      return extract_text_before_comments(news_content)
    else:
      print("Content element not found for url:", url)
  else:
    print("Failed to fetch news content from the provided link")

def extract_text_before_comments(text):
    # Define a pattern to find "Comments" in a contextually irrelevant position (case-insensitive)
    pattern = re.compile(r"comments", re.IGNORECASE)
    # Find the index where "Comments" appears in the text
    match = re.search(pattern, text)
    if match:
        # Extract text before the occurrence of "Comments"
        filtered_text = text[:match.start()].strip()
        return filtered_text
    else:
        return "No 'Comments' found or improperly formatted text"

In [33]:
pyq_Df = pd.read_csv('/content/drive/MyDrive/ColabData/pyq_with_embedding.csv')
pyq_Df['embedding'] = pyq_Df['embedding'].apply(eval).apply(np.array ) #to be used for Numpy calculation

In [34]:
import feedparser
import os
from datetime import datetime, timedelta

def get_valid_date_input():
    while True:
        user_input = input("Enter a date in YYYY-MM-DD format (press Enter for today's date): ")
        if not user_input:  # Check if the user pressed Enter without providing a date
            return datetime.now().date()  # Return today's date as default
        try:
            date_object = datetime.strptime(user_input, "%Y-%m-%d")
            return date_object.date()
        except ValueError:
            print("Invalid date format. Please enter date in YYYY-MM-DD format.")


def fetch_news_from_feed(rss_url,user_date):
    # Initialize empty lists to store data
    titles = []
    links = []
    subHeading = []
    news_contents = []
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    for entry in feed.entries:
        published_date = datetime.strptime(entry.published, "%a, %d %b %Y %H:%M:%S %z").date()
        if  published_date == user_date:
            titles.append(entry.title)
            links.append(entry.link)
            if hasattr(entry, 'description') and entry.description:
                  subHeading.append(entry.description)
            else:
                subHeading.append(entry.summary)
            news_contents.append(get_news_content(entry.link))
    data = {
    'Title': titles,
    'Link': links,
    'SubHeading': subHeading,
    'News Content': news_contents
        }
    return pd.DataFrame(data)


all_news_articles_Df = pd.DataFrame()
# List of RSS feed URLs
rss_feeds = [
    'https://www.thehindu.com/feeder/default.rss',
    'https://www.thehindu.com/sci-tech/science/feeder/default.rss',
    'https://www.thehindu.com/business/agri-business/feeder/default.rss',
    'https://www.thehindu.com/business/Industry/feeder/default.rss',
    'https://www.thehindu.com/business/Economy/feeder/default.rss',
    'https://www.thehindu.com/business/budget/feeder/default.rss',
    'https://www.thehindu.com/news/national/feeder/default.rss',
    'https://www.thehindu.com/news/international/feeder/default.rss',
    'https://www.thehindu.com/news/states/feeder/default.rss',
    'https://www.thehindu.com/opinion/columns/feeder/default.rss',
    'https://www.thehindu.com/opinion/editorial/feeder/default.rss',
    'https://www.thehindu.com/opinion/lead/feeder/default.rss'
    # Add more feed URLs here...
]

csv_file_path = '/content/sample_data/todays_epaper.csv'
#Get Data from user input


# Usage
#user_date = get_valid_date_input()
# Get the current date
current_date = datetime.now().date()

# Subtract one day to get the previous date
user_date = current_date - timedelta(days=1)
#user_date = datetime.now().date()
print("User input date:", user_date)

# Check if the file exists
if not os.path.exists(csv_file_path):
  # Fetch news from each RSS feed
     for rss_url in rss_feeds:
         print(f"Fetching news from {rss_url}")
         news_from_rss_feed_Df = fetch_news_from_feed(rss_url,user_date)
         all_news_articles_Df = pd.concat([all_news_articles_Df, news_from_rss_feed_Df], ignore_index=True)
     #print(all_news_articles_Df)
     all_news_articles_Df['Combined_Text'] = all_news_articles_Df['Title'] + '-' + all_news_articles_Df['SubHeading'].astype(str)
     all_news_articles_Df['embedding'] = all_news_articles_Df['Combined_Text'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
     all_news_articles_Df.to_csv(csv_file_path)
     print("File saved successfully.")
else:
    print("File already exists. Skipping the read operation.")

all_news_articles_Df = pd.read_csv(csv_file_path)
all_news_articles_Df['embedding'] = all_news_articles_Df['embedding'].apply(eval).apply(np.array )
#print(all_news_articles_Df)


User input date: 2024-07-24
Fetching news from https://www.thehindu.com/feeder/default.rss
Fetching news from https://www.thehindu.com/sci-tech/science/feeder/default.rss
Fetching news from https://www.thehindu.com/business/agri-business/feeder/default.rss
Fetching news from https://www.thehindu.com/business/Industry/feeder/default.rss
Fetching news from https://www.thehindu.com/business/Economy/feeder/default.rss
Fetching news from https://www.thehindu.com/business/budget/feeder/default.rss
Content element not found for url: https://www.thehindu.com/news/national/parliament-monsoon-session-budget-2024-live-updates-july-24/article68440069.ece
Fetching news from https://www.thehindu.com/news/national/feeder/default.rss
Fetching news from https://www.thehindu.com/news/international/feeder/default.rss
Content element not found for url: https://www.thehindu.com/shorts/watch-us-election-2024-foreign-policy-implications-that-could-impact-india/article68441276.ece
Content element not found fo

In [35]:
#finding news relevant for syllabus and PYQ
from google.colab import files
permanent_file_path = '/content/drive/MyDrive/ColabData/EditorialScitechCombined/compilation.xlsx'
final_news = []
ed_sci_news = []
news_temp = []
print(user_date)
user_input_num = "0.82"  #defaulting to 0.82, you can also take user inputs here
user_input_num = (
    round(float(user_input_num.strip() or "0.82"), 3)
    if (user_input_num.strip() or "0").replace(".", "", 1).isdigit()
    and 0.80 <= float(user_input_num.strip() or "0") <= 1
    else 0.82  # Default value if conditions are not met
)
print(user_input_num)

all_news_articles_Df.drop_duplicates(subset=['Link'], inplace=True) #Removing duplicate links
for index, line in all_news_articles_Df.iterrows():
    summary_text = None
    filtered_Df = pyq_Df[
        pyq_Df["embedding"].apply(
            lambda x: cosine_similarity(x, line["embedding"]) > user_input_num
        )
    ]
    if not filtered_Df.empty:
        # Calculate similarity scores for the filtered entries
        filtered_Df = filtered_Df.copy()  # Explicitly create a copy
        filtered_Df['similarity'] = filtered_Df["embedding"].apply(
            lambda x: cosine_similarity(x, line["embedding"])
        )
        filtered_Df = filtered_Df.sort_values(by='similarity', ascending=False)
        most_relevant_pyq = '\n'.join(filtered_Df['Questions'])
        try:
         news_temp.append(
             {
                 "Link": line["Link"],
                 "News Content": openai_generate_summary(line["News Content"]), #line["News Content"],
                 "Relevant PYQ": most_relevant_pyq,
                 "Title": line["Title"],
                 'SubHeading': line['SubHeading'],
                 "Date": user_date
             }
            )
        except Exception as e:#
         news_temp.append(
             {
                 "Link": line["Link"],
                 "News Content": line["News Content"], #line["News Content"],
                 "Relevant PYQ": most_relevant_pyq,
                 "Title": line["Title"],
                 'SubHeading': line['SubHeading'],
                 "Date": user_date
             }
            )

#news_temp_Df = pd.DataFrame(
 #   news_temp, columns=["Link", "News Content", "Relevant PYQ", "Title", "SubHeading"]
#)
#total_records = len(news_temp_Df)
final_news_Df = pd.DataFrame(news_temp, columns=['Link','SubHeading','Relevant PYQ','News Content'])
final_news_Df.to_csv('/content/sample_data/relevant_news_link.csv', index=False)
files.download('/content/sample_data/relevant_news_link.csv')

2024-07-24
0.82


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Future project
1.   Use Flask for interactive user inteface
2.   Use Google cloud scheduler to run notebook daily at specific time

